# Optimization Attempt #1 includes:

1.   Dropping the STATUS and SPECIAL_CONSIDERATIONS columns as they may have less relevance on the success of a project.  
2.   Changing the APPLICATION_TYPE 'Other' bin to include value_counts less than 10.
3.   Changing the CLASSIFICATION 'Other' bin to include value_counts less than 100.

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Try dropping the columns 'STATUS' and 'SPECIAL_CONSIDERATIONS', as they may not have as much relevance to success.
application_df = application_df.drop(columns=['STATUS', 'SPECIAL_CONSIDERATIONS'])

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = application_type_counts[application_type_counts < 10].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_value_counts = application_df['CLASSIFICATION'].value_counts()


In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_greater_than_one = classification_value_counts[classification_value_counts>1]
classification_greater_than_one

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_value_counts[classification_value_counts < 100].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
category_dummies = pd.get_dummies(application_df)

In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = category_dummies.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  6
hidden_nodes_layer2 = 6

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 306       
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 355 (1.39 KB)
Trainable params: 355 (1.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
fit_model = nn1.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.6209 - accuracy: 0.6769
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5721 - accuracy: 0.7201
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5617 - accuracy: 0.7240
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5574 - accuracy: 0.7257
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5551 - accuracy: 0.7260
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5535 - accuracy: 0.7283
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5526 - accuracy: 0.7270
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5517 - accuracy: 0.7276
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5513 - accuracy: 0.7289
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5506 - accura

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5540 - accuracy: 0.7289 - 751ms/epoch - 3ms/step
Loss: 0.5540290474891663, Accuracy: 0.728863000869751


# Optimization Attempt #1 results:

Dropping the STATUS and SPECIAL_CONSIDERATIONS columns, changing the APPLICATION_TYPE 'Other' bin, and changing the CLASSIFICATION 'Other' bin had no effect on the outcome.  The loss is still approximately .55 and the accuracy .73, as they were in the original attempt.

# Optimization Attempt #2 includes:

1.   The same dropped columns and bin changes as attempt #1
2.   Increasing the number of nodes for both hidden layers 1 & 2.
3.   Adding a third hidden layer with 4 nodes.

In [17]:
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df = application_df.drop(columns=['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'])

# Bin less used application types into "Other"
application_types_to_replace = application_type_counts[application_type_counts < 10].index.tolist()
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Bin less frequent classifications into "Other"
classifications_to_replace = classification_value_counts[classification_value_counts < 100].index.tolist()
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

In [18]:
# Convert categorical data to numeric with `pd.get_dummies`
category_dummies = pd.get_dummies(application_df)

# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = category_dummies.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 4

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 10)                510       
                                                                 
 dense_4 (Dense)             (None, 8)                 88        
                                                                 
 dense_5 (Dense)             (None, 4)                 36        
                                                                 
 dense_6 (Dense)             (None, 1)                 5         
                                                                 
Total params: 639 (2.50 KB)
Trainable params: 639 (2.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
fit_model = nn2.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.6091 - accuracy: 0.6754
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5605 - accuracy: 0.7258
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5551 - accuracy: 0.7280
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5533 - accuracy: 0.7294
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7290
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5511 - accuracy: 0.7294
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5500 - accuracy: 0.7308
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7301
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5487 - accuracy: 0.7302
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accura

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5541 - accuracy: 0.7280 - 483ms/epoch - 2ms/step
Loss: 0.5540838241577148, Accuracy: 0.7280466556549072


# Optimization Attempt #2 results:

Adding additional nodes to the existing two layers, and a third layer of 4 nodes also had no effect on the outcome.  The loss is still approximately .55 and the accuracy .73, as they were in the original attempt and the first optimization attempt.

# Optimization Attempt #3 includes:

1.   The same dropped columns and bin changes as attempt #1 & 2 plus USE_CASE and ORGANIZATION.
2.   Additional increases to nodes in layers 1 - 3.
3.   Adding a fourth hidden layer with 4 nodes and an activation type of Tanh.
4.   Reducing Epochs to 50 as all previous attempts stabilize quickly.




In [73]:
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df = application_df.drop(columns=['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS', 'USE_CASE', 'ORGANIZATION',])

# Bin less used application types into "Other"
application_types_to_replace = application_type_counts[application_type_counts < 10].index.tolist()
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Bin less frequent classifications into "Other"
classifications_to_replace = classification_value_counts[classification_value_counts < 100].index.tolist()
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

In [74]:
# Convert categorical data to numeric with `pd.get_dummies`
category_dummies = pd.get_dummies(application_df)

# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = category_dummies.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [75]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 32
hidden_nodes_layer2 = 16
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 4

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_52 (Dense)            (None, 32)                1344      
                                                                 
 dense_53 (Dense)            (None, 16)                528       
                                                                 
 dense_54 (Dense)            (None, 8)                 136       
                                                                 
 dense_55 (Dense)            (None, 4)                 36        
                                                                 
 dense_56 (Dense)            (None, 1)                 5         
                                                                 
Total params: 2049 (8.00 KB)
Trainable params: 2049 (8.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [76]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [77]:
fit_model = nn3.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 2ms/step - loss: 0.5800 - accuracy: 0.7186
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5634 - accuracy: 0.7269
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5620 - accuracy: 0.7262
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5602 - accuracy: 0.7273
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5595 - accuracy: 0.7269
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5585 - accuracy: 0.7278
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5580 - accuracy: 0.7288
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5574 - accuracy: 0.7278
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5568 - accuracy: 0.7290
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5574 - accuracy: 0.7272

In [78]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5619 - accuracy: 0.7261 - 489ms/epoch - 2ms/step
Loss: 0.561870276927948, Accuracy: 0.726064145565033


# Optimization Attempt #3 results:

Further reducing columns considered, adding additional nodes to the existing three layers, and a fourth layer of 8 nodes, and using an additional activation type, all had no effect on the outcome.  The loss is very slightly worse at .56 and the accuracy also nearly the same at .73, as they were in the original attempt and the first and second optimization attempts. Tinkering with included columns yielded that Affiliation is an important feature.  Removing it leads to worse loss (.63) and worse accuracy (.63).

Original Attempt:  Loss: 0.554374098777771, Accuracy: 0.72967928647995

Optimization Attempt #1:  Loss: 0.5560629963874817, Accuracy: 0.7274635434150696

Optimization Attempt #2:  Loss: 0.5528214573860168, Accuracy: 0.7282798886299133

Optimization Attempt #3:  Loss: 0.5623424649238586, Accuracy: 0.7268804907798767

Attempt #3 w/o 'AFFILIATION':  Loss: 0.6336261630058289, Accuracy: 0.6261224746704102

In [47]:
# Export best model to HDF5 file
nn2.save('AlphabetSoupCharity_Optimization.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
